# Web Scraping for Collecting Airport Data

## Introduction:

This Jupyter Notebook focuses on collecting data about airports in Central America using web scraping from Wikipedia. The objective is to gather essential information about airport locations, names, and other relevant details.

## Objectives:

1. Obtain data on airports in Central America from Wikipedia.
2. Extract key information such as airport names, locations, and other relevant details.
3. Store the collected data in a suitable format for further analysis and use in our database.

## Libraries

* **Requests**: The requests library allows users to make HTTP requests to web pages, facilitating the download of HTML content from Wikipedia for further processing.

* **Beautiful Soup (bs4)**: Beautiful Soup is a useful tool for parsing and searching HTML elements in the downloaded content. It enables users to extract specific information from Wikipedia pages, such as titles, paragraphs, links, and more.

* **Pandas**: Pandas is an essential library for structuring and manipulating extracted data. It allows users to organize data into rows and columns, facilitating operations such as cleaning, filtering, and processing.


In [5]:
import requests
import bs4
import requests
import pandas as pd
import re

In [6]:
def get_airport():
    
    url = "https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_Central_America"

    response = requests.get(url)

    soup = bs4.BeautifulSoup(response.text, 'html.parser')

    
    airport_table = soup.find_all('table')[3]

    
    data = {"Country": [], "Airport Name": [], "IATA Code": [],"ICAO Code":[], "City Served": [], "Passengers": []}

    
    for row in airport_table.find_all('tr')[1:]:
        columns = row.find_all('td')
        data["Country"].append(columns[1].get_text(strip=True))
        data["Airport Name"].append(columns[2].get_text(strip=True))
        data["IATA Code"].append(columns[3].get_text(strip=True).split('/')[0])
        data["ICAO Code"].append(columns[3].get_text(strip=True).split('/')[1])
        data["City Served"].append(columns[4].get_text(strip=True))
        data["Passengers"].append(int(columns[5].get_text(strip=True).replace(",", "").split('[')[0]))
        

    df = pd.DataFrame(data)
                
    return df

In [7]:
df = get_airport()

In [8]:
def get_coords(df):
    
    coordinates = {"Lat": [], "Lon": []}

    for airport_extension in df.iloc[:,1]:

        if airport_extension == 'Albrook International Airport':

            url = "https://en.wikipedia.org/wiki/Albrook_%22Marcos_A._Gelabert%22_International_Airport"

        else:

            url = f"https://en.wikipedia.org/wiki/{airport_extension.replace(' ','_')}"

        response = requests.get(url)

        soup = bs4.BeautifulSoup(response.text, 'html.parser')

        coords_table = soup.find('table',{'class':'infobox vcard'})

        coords = coords_table.find('span', class_='geo-inline').get_text(strip=True).split('\ufeff /')

        lat = coords[-1].split(';')[0]
        lon = coords[-1].split(';')[1]

        coordinates["Lat"].append(lat)
        coordinates["Lon"].append(lon)

    df["Lat"] = coordinates.get("Lat")
    df["Lon"] = coordinates.get("Lon")
    
    return df

In [9]:
df = get_coords(df)

df.head()

,Country,Airport Name,IATA Code,ICAO Code,City Served,Passengers,Lat,Lon
0,Panama,Tocumen International Airport,PTY,MPTO,Panamá City,14741937,9.07139,-79.38361
1,Costa Rica,Juan Santamaría International Airport,SJO,MROC,San José,6456750,9.99389,-84.20889
2,El Salvador,Comalapa International Airport,SAL,MSLP,San Salvador,2984764,13.44083,-89.05556
3,Guatemala,La Aurora International Airport,GUA,MGGT,Guatemala City,2579123,14.58167,-90.52667
4,Nicaragua,Augusto C. Sandino International Airport,MGA,MNMG,Managua,1533034,12.14139,-86.16806


In [10]:
df.to_csv('Airports_Data.csv')